In [ ]:
import requests
from dotenv import load_dotenv

In [ ]:
load_dotenv()

user_preferences = {
    "interests": ["Technology", "Science", "Art"],
    "content_types": ["Article", "Tutorial"],
    "text_format": "summary"
}

mock_content = [
    {"title": "The Future of Tech: Innovations to Watch", "category": "Technology", "type": "Article", "summary": "Explore the cutting-edge technologies shaping our future, from AI advancements to sustainable energy solutions."},
    {"title": "Beginner's Guide to Digital Art", "category": "Art", "type": "Tutorial", "summary": "Step into the world of digital art with this comprehensive tutorial for beginners. Learn about tools, techniques, and tips to start your digital art journey."},
    {"title": "Understanding Quantum Computing", "category": "Science", "type": "Article", "summary": "Dive into the complex world of quantum computing and discover how it's set to revolutionize technology with unparalleled computational power."},
    {"title": "The Renaissance Art Movement", "category": "Art", "type": "Summary", "summary": "A brief overview of the Renaissance period, highlighting its most influential artists and their timeless masterpieces."},
    {"title": "DIY Home Robotics Projects", "category": "Technology", "type": "Tutorial", "summary": "Get hands-on with technology by exploring these simple yet fascinating DIY robotics projects you can build at home."},
    {"title": "The Mysteries of Dark Matter", "category": "Science", "type": "Article", "summary": "Join us on a journey through the cosmos as we explore the enigmatic nature of dark matter and its role in the universe."},
    {"title": "Creative Writing Tips for Beginners", "category": "Art", "type": "Tutorial", "summary": "Unlock your creative potential with these essential writing tips, perfect for aspiring authors starting their journey."},
    {"title": "Innovative Eco-Friendly Technologies", "category": "Technology", "type": "Summary", "summary": "Discover how innovative technologies are paving the way for a more sustainable future, focusing on renewable energy sources and eco-friendly solutions."},
    {"title": "Exploring the Human Genome", "category": "Science", "type": "Summary", "summary": "An insightful summary of what the human genome project has taught us and how it continues to influence scientific research and medical advancements."},
    {"title": "Photography Basics: Capturing the Moment", "category": "Art", "type": "Article", "summary": "Learn the fundamentals of photography with this guide, from understanding your camera to composing the perfect shot."},
]


In [ ]:
def set_user_preferences(interests, content_types):
    return {
        "interests": interests,
        "content_types": content_types
    }

# Example of setting preferences
user_preferences = set_user_preferences(["Technology", "Art"], ["Article", "Tutorial"])

print(user_preferences)


In [ ]:


def generate_content(preferences):
    # Filter content based on user preferences
    filtered_content = [content for content in mock_content if content["category"] in preferences["interests"] and content["type"] in preferences["content_types"]]
    return filtered_content

# Example usage
personalized_content = generate_content(user_preferences)
print(personalized_content)


In [ ]:
from openai import OpenAI

client = OpenAI()

stream = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
            {"role": "system", "content": "You are an AI who writes a summary of the article."},
            {"role": "user", "content": "Say this is a test"},
        ]
        ,
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

# LANGCHAIN PROTOTYPE

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import os

In [ ]:
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))


def generate_content_with_langchain(interest, content_type):
    # Craft a prompt based on user interest and desired content type
    template = """Question: {question}

    Answer: Let's think step by step."""

    prompt = PromptTemplate.from_template(template)
    question_template = """Create a {content_type} about {interest}. 
    It should be engaging and informative."""
    question = question_template.format(content_type=content_type, interest=interest)
    # Use Langchain's OpenAI LLM to generate content
    # Note: Adjust the parameters according to your requirements
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    # Extracting the generated content
    generated_content = llm_chain.run(question)

    return {
        "title": f"{interest} {content_type}",
        "content": generated_content
    }

# Example usage
interest = "Sustainable Living"
content_type = "Summary"
generated_content = generate_content_with_langchain(interest, content_type)
print(generated_content)

In [ ]:
print(generated_content["content"])

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("where did harrison work?")

# TruLens

In [ ]:
from dotenv import load_dotenv
load_dotenv()
# Imports main tools:
from trulens_eval import TruChain, Feedback, Tru
tru = Tru()
tru.reset_database()

# Imports from langchain to build app
import bs4
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

splits = text_splitter.split_documents(docs)

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings()
)

In [ ]:
# RAG

retriever = vectorstore.as_retriever()

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("What is Task Decomposition?")

In [ ]:
from trulens_eval.feedback.provider import OpenAI
import numpy as np

# Initialize provider class
openai = OpenAI()

# select context to be used in feedback. the location of context is app specific.
from trulens_eval.app import App
context = App.select_context(rag_chain)

from trulens_eval.feedback import Groundedness
grounded = Groundedness(groundedness_provider=OpenAI())
# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons)
    .on(context.collect()) # collect context chunks into a list
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on_input_output()
# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(openai.qs_relevance)
    .on_input()
    .on(context)
    .aggregate(np.mean)
)

In [ ]:
tru_recorder = TruChain(rag_chain,
    app_id='Chain1_ChatApplication',
    feedbacks=[f_qa_relevance, f_context_relevance, f_groundedness])

In [ ]:
with tru_recorder as recording:
    llm_response = rag_chain.invoke("What is Task Decomposition?")

display(llm_response)

In [ ]:
# The record of the app invocation can be retrieved from the `recording`:

rec = recording.get() # use .get if only one record
# recs = recording.records # use .records if multiple

display(rec)

In [ ]:
# The results of the feedback functions can be rertireved from
# `Record.feedback_results` or using the `wait_for_feedback_result` method. The
# results if retrieved directly are `Future` instances (see
# `concurrent.futures`). You can use `as_completed` to wait until they have
# finished evaluating or use the utility method:

for feedback, feedback_result in rec.wait_for_feedback_results().items():
    print(feedback.name, feedback_result.result)

# See more about wait_for_feedback_results:
# help(rec.wait_for_feedback_results)

In [ ]:
records, feedback = tru.get_records_and_feedback(app_ids=["Chain1_ChatApplication"])

records.head()

In [ ]:
tru.get_leaderboard(app_ids=["Chain1_ChatApplication"])

In [ ]:
tru.run_dashboard()

In [ ]:
tru.stop_dashboard()

In [ ]:
from langchain_exa import ExaSearchRetriever, TextContentsOptions
# This is for the exa search retriever